# Random walk

In [ ]:
#| default_exp graph.random_walk

In [ ]:
#| export
import scipy.sparse as sp, numpy as np

from sklearn.preprocessing import normalize
from xclib.utils import sparse as xs
from numba import njit, prange
from xclib.utils import graph
from tqdm.auto import tqdm


In [ ]:
#| export
def normalize_graph(mat):
    col_nnz = np.sqrt(1/(np.ravel(mat.sum(axis=0))+1e-6))
    row_nnz = np.sqrt(1/(np.ravel(mat.sum(axis=1))+1e-6))
    c_diags = sp.diags(col_nnz)
    r_diags = sp.diags(row_nnz)
    mat = r_diags.dot(mat).dot(c_diags)
    mat.eliminate_zeros()
    return mat
    

In [ ]:
#| export
def clean_graphs(min_freq=2, max_freq=100, *graphs):
    smats = sp.vstack([graph.data for graph in graphs], "csr")
    node_freq = np.ravel(smats.getnnz(axis=0))
    num_nodes = node_freq.size
    keep_nodes = np.ones(num_nodes)
    keep_nodes[node_freq > max_freq] = 0
    keep_nodes[node_freq <= min_freq] = 0
    diag = sp.diags(keep_nodes, shape=(num_nodes, num_nodes))
    for i in range(len(graphs)):
        graphs[i].data = graphs[i].data.dot(diag)
        graphs[i].data.eliminate_zeros()
    return graphs
    

In [ ]:
#| export
def Prune(G, R, C, batch_size=1024, normalize=True):
    R = normalize(R)
    C = normalize(C)
    rows, cols = G.nonzero()
    _nnz = G.nnz
    data = np.zeros(_nnz)
    for start in tqdm(np.arange(0, _nnz, batch_size), desc="Pruning"):
        end = min(start + batch_size, _nnz)
        _R = R[rows[start:end]]
        _C = C[cols[start:end]]
        data[start:end] = np.ravel(np.sum(_R*_C, axis=1))
    data[data < 0] = 0
    OG = sp.csr_matrix((data, (rows, cols)), shape=G.shape)
    OG.eliminate_zeros()
    csr_stats(OG, "GRAPH")
    if normalize:
        OG = normalize_graph(OG)
    return OG
    

In [ ]:
#| export
def print_stats(mat, k=10):
    _mat = mat.copy()
    _mat.__dict__['data'][:] = 1
    freqs = _mat.sum(axis=1)
    print(np.max(freqs), np.min(freqs), np.std(freqs))
    

## `Random walk`

In [ ]:
#| export
@njit(parallel=True, nogil=True)
def _random_walk(data_lbl_indices:np.ndarray, data_lbl_indptr:np.ndarray, lbl_data_indices:np.ndarray, lbl_data_indptr:np.ndarray, 
                 walk_to:int, p_reset:float, hops_per_step:int, start:int, end:int):
    n_data = end - start
    nbr_idx = np.zeros((n_data, walk_to), dtype=np.int32)
    nbr_data = np.zeros((n_data, walk_to), dtype=np.float32)
    
    for idx in range(0, n_data):
        for walk in np.arange(0, walk_to):
            p = np.random.random()
            if walk == 0 or p < p_reset: data_i = idx + start
        
            data_start, data_end = data_lbl_indptr[data_i], data_lbl_indptr[data_i+1]
            if data_start - data_end == 0: continue
                
            lbl_i = np.random.choice(data_lbl_indices[data_start: data_end])
            
            lbl_start, lbl_end = lbl_data_indptr[lbl_i], lbl_data_indptr[lbl_i+1]
            data_i = np.random.choice(lbl_data_indices[lbl_start: lbl_end])
            
            nbr_idx[idx, walk] = data_i if hops_per_step == 2 else lbl_i
            nbr_data[idx, walk] = 1
            
    return nbr_idx.flatten(), nbr_data.flatten()


In [ ]:
#| export
class PrunedWalk(graph.RandomWalk):
    
    def __init__(self, data_lbl):
        self.data_lbl = data_lbl.tocsr()
        self.data_lbl.sort_indices()
        self.data_lbl.eliminate_zeros()

    def simulate(self, walk_to=100, p_reset=0.2, k=None, hops_per_step=2, b_size=1000):
        assert hops_per_step == 1 or hops_per_step == 2, f"Invalid hops per step: {hops_per_step}"
        
        data_lbl_indices = self.data_lbl.indices
        data_lbl_indptr = self.data_lbl.indptr
        
        lbl_data = self.data_lbl.transpose().tocsr()
        lbl_data.sort_indices()
        lbl_data.eliminate_zeros()
        lbl_data_indices = lbl_data.indices
        lbl_data_indptr = lbl_data.indptr

        n_data = self.data_lbl.shape[0]
        n_lbl = self.data_lbl.shape[hops_per_step % 2]
        
        walks = list()
        for idx in tqdm(range(0, n_data, b_size)):
            start, end = idx, min(idx+b_size, n_data)
            cols, data = _random_walk(data_lbl_indices, data_lbl_indptr, lbl_data_indices, lbl_data_indptr, 
                                      walk_to, p_reset, hops_per_step, start=start, end=end)
            
            rows = np.arange(end-start).reshape(-1, 1)
            rows = np.repeat(rows, walk_to, axis=1).flatten()
            
            walk = sp.coo_matrix((data, (rows, cols)), dtype=np.float32, shape=(end-start, n_lbl))
            walk.sum_duplicates()
            walk = walk.tocsr()
            walk.sort_indices()
            
            if k is not None: walk = xs.retain_topk(walk, k=k).tocsr()
                
            walks.append(walk)
            del rows, cols
            
        return sp.vstack(walks, "csr")


In [ ]:
#| export
def remove_cols(matrix, idx):
    num_cols = matrix.shape[1]
    col_mask = np.ones(num_cols, dtype=np.int32)
    col_mask[idx] = 0
    mask = sp.diags(col_mask, shape=(num_cols, num_cols))
    return matrix.dot(mask).tocsr()


In [ ]:
#| export
def remove_rows(matrix, idx):
    num_rows = matrix.shape[0]
    row_mask = np.ones(num_rows, dtype=np.int32)
    row_mask[idx] = 0
    mask = sp.diags(row_mask, shape=(num_rows, num_rows))
    return mask.dot(matrix).tocsr()
    

In [ ]:
#| export
def random_walk(matrix, row_head_thresh=500, col_head_thresh=500, walk_length=400, p_reset=0.8, topk=10, batch_size=1023):
    matrix = matrix.tocsr()
    
    idxs = np.where(matrix.getnnz(axis=1) > row_head_thresh)[0]
    pruned_matrix = remove_rows(matrix, idxs) if len(idxs) > 0 else matrix
        
    idxs = np.where(matrix.getnnz(axis=0) > col_head_thresh)[0]
    if len(idxs) > 0: pruned_matrix = remove_cols(pruned_matrix, idxs)
    
    return PrunedWalk(pruned_matrix).simulate(walk_length, p_reset, topk, 2, batch_size)
    

In [ ]:
from scipy.sparse.csgraph import connected_components

In [ ]:
data_dir = '/home/scai/phd/aiz218323/scratch/outputs/mogicX/01-msmarco-llama-entity-linker-002/predictions/'
matrix = sp.load_npz(f'{data_dir}/test_predictions_full.npz')

In [ ]:
matrix = sp.csr_matrix([[1, 1, 1, 0, 0, 0, 0, 0], 
                        [1, 1, 0, 0, 0, 0, 0, 0], 
                        [1, 0, 1, 0, 0, 0, 0, 0], 
                        [0, 0, 0, 1, 1, 0, 0, 0], 
                        [0, 0, 0, 1, 1, 0, 1, 0], 
                        [0, 0, 0, 0, 0, 1, 0, 1], 
                        [0, 0, 0, 0, 1, 0, 1, 1], 
                        [0, 0, 0, 0, 0, 1, 1, 1]])

In [ ]:
rnd_mat = random_walk(matrix, row_head_thresh=2, col_head_thresh=2)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
n_components, labels = connected_components(matrix, directed=False, return_labels=True); labels

array([0, 0, 0, 1, 1, 1, 1, 1], dtype=int32)

In [ ]:
n_components, labels = connected_components(rnd_mat, directed=False, return_labels=True); labels

array([0, 0, 0, 1, 1, 1, 1, 1], dtype=int32)

In [ ]:
thresh_mat = rnd_mat.multiply(rnd_mat > 150).todense()
n_components, labels = connected_components(thresh_mat, directed=False, return_labels=True); labels

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int32)

In [ ]:
rnd_mat.todense()

matrix([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0., 400.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0., 400.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0., 400.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0., 400.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]], dtype=float32)